<span style="font-size: 24px;">**Animal detection in a picture using yolov4**</span>

In [ ]:
from glob import glob

In [ ]:
images_paths = glob('./animal*')

In [ ]:
import cv2
import numpy as np
from IPython.display import Image, display

# files related to yolov4
COCO_LABELS_FILE = 'coco.names'  
YOLO_CONFIG_FILE = 'yolov4.cfg'  
YOLO_WEIGHTS_FILE = 'yolov4.weights'

# Loading the image
for image1 in images_paths:
    img = cv2.imread(image1)
    img_copy = img.copy() 
    
    # Loading the labels required for the YOLO model (not used)
    with open(COCO_LABELS_FILE, 'rt') as f:
        labels = f.read().rstrip('\n').split('\n')
    
    # Loading the YOLO model
    yolo = cv2.dnn.readNetFromDarknet(YOLO_CONFIG_FILE, YOLO_WEIGHTS_FILE)
    
    # Resizing the picture for the model
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    yolo.setInput(blob)
    
    layer_names = yolo.getLayerNames()
    output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]
    
    # Detecting the animal
    outs = yolo.forward(output_layers)
    
    # Processing the results
    boxes_detected = []
    confidences_scores = []
    labels_detected = []
    height, width = img.shape[:2]
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
    
            if confidence > 0.3:  #Only keep detections with confidence higher than 0.3
                centerX = int(detection[0] * width)
                centerY = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
    
                # Calculating the top-left corner coordinates
                x = int(centerX - w / 2)
                y = int(centerY - h / 2)
    
                # Add the results to the list
                boxes_detected.append([x, y, w, h])
                confidences_scores.append(float(confidence))
                labels_detected.append(classID)
    
    # Applying Non-Maximum Suppression to remove overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes_detected, confidences_scores, score_threshold=0.3, nms_threshold=0.4)
    
    # Drawing detection boxes and "animal" label on the image
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes_detected[i]
            label = "animal" # single class "animal"
            color = (0, 0, 255)  # Red for bounding boxes
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), color, 2) # Drawing detection boxes
            cv2.putText(img_copy, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Displaying and saving the picture
    output_file = "output_with_animal.jpg"
    cv2.imwrite(output_file, img_copy) # saving 
    display(Image(output_file, width=512, height=512))

